In [ ]:
import numpy as np


class Kernel:

    @staticmethod
    def polynomialKernel(A, degree=2, tildeA=np.array([]), c=1):        
        if tildeA.size > 0:
            K = A.dot(tildeA.T) # Compute the dot product between each pair of rows from A and tildeA
            K = (K + c) ** degree # Compute the polynomial kernel
        else:
            temp = A.dot(A.T) 
            K = (temp + c) ** degree 
        
        return K


    @staticmethod
    def rbfKernel(A, gamma, tildeA=np.array([]), degree=None):
        ''' 
        Build gaussian kernel matrix with tildaA:

        gamma = width parameter; kernel value: exp(-gamma(Ai-Aj)^2)
        A = full data set
        tildeA = can be full or reduced set # REDUCED SVM
        
        Output:
        K = kernel data using Gaussian kernel
        '''

        if tildeA.size > 0:
            # If dimensions of A is not equal to tildeA, then return error
            AA = np.kron(np.ones((1, tildeA.shape[0])),np.sum(A**2, axis=1).reshape(A.shape[0], 1),)
            tildeAA = np.kron(np.ones((1, A.shape[0])),np.sum(tildeA**2, axis=1).reshape(tildeA.shape[0], 1),)
            K = np.exp((-AA - tildeAA.T + 2 * A.dot(tildeA.T))* gamma)
        else:
            AA = np.kron(np.ones((1, A.shape[0])),np.sum(A**2, axis=1).reshape(A.shape[0], 1),)
            K = np.exp((-AA - AA.T + 2 * A.dot(A.T)) * gamma)
            
        return K


    @staticmethod
    def buildKernel(params, gamma, A, tildeA=np.array([]), degree=None, c=None):
        ''' 
        Build kernel data matrix, no matter matrix is full or reduced
        
        params = 0 - linear, 1-rbf
        A = A is a [m x n] real number matrix
        tildeA = a [p x n] real number matrix
        gamma = kernel arguments(it dependents on your kernel type)
        
        Output:
        K = flag -> indicate which type is
            Kernel -> kernel matrix
        '''
        
        # Polynomial Kernel
        if params == 2:
            K = {"flag": "dual", "Kernel": Kernel.polynomialKernel(A, degree, tildeA, c)} 

        # RBF Kernel
        elif params == 1:
            K = {"flag": "dual", "Kernel": Kernel.rbfKernel(A, gamma, tildeA)}

        # Linear Kernel
        elif params == 0:
            if A.shape[1] > A.shape[0] and tildeA.size > 0:
                try:
                    K = {"flag": "dual", "Kernel": np.dot(A, tildeA.T)}
                except:
                    print("\n===Error in buildKernel : A index must equal to tildeA===")
                    return None
            else:
                K = {"flag": "primal", "Kernel": A}
        else:
            K = {"flag": "dual", "Kernel": A}

        return K